In [1]:
# 处理数据示例
import pandas as pd
from fin_data_processor import FinDataProcessor
processor = FinDataProcessor(data_dir='./data')
df = pd.read_feather("output/continuous_IF_volume5d.feather")
# df = processor.load_data(start_date='2024-04-22', end_date='2024-11-18')
df = processor.clean_data(df)
df = processor.add_calendar_features(df)
df = processor.add_features(df)
#删除掉vol列为空的行
df = df[df['vol'].notna()]
df = df.reset_index()
# df = processor.add_features(df,'spread')
df.to_feather('data.feather')

In [ ]:
import pandas as pd
# import factor_compute, factor_register, factor_manager, factors_test
# factor_register.register_all_factors()
# df = pd.read_feather('data.feather')
# factor_manager.FactorManager.get_factor_info()
# custom_config = factors_test.FactorTestConfig(
#         ic_method='spearman',
#         ic_threshold=0.03,
#         ir_threshold=0.6
#     )
# 测试单个因子
# 批量测试所有新因子
# new_factors = [
#  "microstructure_momentum"
# ]

# for factor in new_factors:
#     results = factor_compute.run_factor_compute(df, factor_name=factor)
# 测试单个因子
# 测试多个因子
# factor_list = ["time_decay", "term_premium", "volume_price_trend"]
# for factor in factor_list:
#     results = factor_compute.run_factor_compute(df, factor_name=factor)


In [ ]:
import pandas as pd
from factor_compute import *
from factor_register import *
from factor_manager import *
from factors_test import *

df = pd.read_feather("data.feather")
Manager = FactorManager()
register_all_factors()
Manager.get_factor_info()


In [ ]:
import pandas as pd
from factor_compute import *
from factor_register import *
from factor_manager import *
from factors_test import *

df = pd.read_feather("data.feather")
Manager = FactorManager()
register_all_factors()
factors_list = Manager.get_factor_names(frequency=FactorFrequency.TICK)

custom_config = FactorTestConfig(
    ic_method='spearman',
    ic_threshold=0.03,
    ir_threshold=0.6
)

for factor in factors_list:
    results = run_factor_compute(df, 
                               factor_name=factor,
                               config=custom_config,
                               return_periods=[10],
                               generate_images=False)
print(results['summary'])



In [ ]:
import pandas as pd
from factor_compute import run_factor_compute, compute_multiple_factors,compute_factors_in_chunks
from factors_test import FactorTestConfig
from factor_register import *
from factor_manager import *

# 加载数据
df = pd.read_feather("data.feather")
Manager = FactorManager()
register_all_factors()

# 获取因子名称
factor_to_compute = Manager.get_factor_names(frequency=FactorFrequency.TICK)
print("开始分块计算因子...")

# 使用修改后的函数计算
df_with_factors = compute_factors_in_chunks(
    df=df,
    factor_names=factor_to_compute,
    chunk_size=1000000,  # 每块100万行
    overlap_size=1200,   # 重叠区域设为最大窗口大小
    adjust_sign=True
)

# 保存结果
df_with_factors.to_feather('data_with_factors.feather')
print("\n因子计算完成，结果已保存")

开始分块计算因子...
数据集总行数: 17957706
分块大小: 1000000, 重叠区域: 1200
需要处理的块数: 18

处理第 1/18 块数据 (索引 0 到 999999), 大小: 1000000 行

开始计算 31 个因子...

开始计算因子: momentum_10
数据列名: ['index', 'TradDay', 'UpdateTime', 'InstruID', 'LastPrice', 'HighPrice', 'LowPrice', 'OpenPrice', 'Volume', 'LastVolume', 'Turnover', 'OpenInt', 'PreOpenInt', 'OpenIntChg', 'ClosePrice', 'SetPrice', 'PreSetPrice', 'PreCloPrice', 'BuyVolume', 'SellVolume', 'AvgBuyPrice', 'AvgSellPrice', 'BidPrice1', 'BidPrice2', 'BidPrice3', 'BidPrice4', 'BidPrice5', 'AskPrice1', 'AskPrice2', 'AskPrice3', 'AskPrice4', 'AskPrice5', 'BidVolume1', 'BidVolume2', 'BidVolume3', 'BidVolume4', 'BidVolume5', 'AskVolume1', 'AskVolume2', 'AskVolume3', 'AskVolume4', 'AskVolume5', 'DerBidVolume1', 'DerBidVolume2', 'DerBidVolume3', 'DerBidVolume4', 'DerBidVolume5', 'DerAskVolume1', 'DerAskVolume2', 'DerAskVolume3', 'DerAskVolume4', 'DerAskVolume5', 'ULimitPrice', 'LLimitPrice', 'InstruCode', 'contract_id', 'date', 'contract_switch', 'DateTime', 'hour', 'minute', 's

/Users/okko_dyd/Desktop/富国实习/factor_register.py:501: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  result['minute'] = pd.to_datetime(result['UpdateTime']).dt.minute



因子计算完成!
因子 intraday_seasonality 计算完成

开始计算因子: term_premium
数据列名: ['index', 'TradDay', 'UpdateTime', 'InstruID', 'LastPrice', 'HighPrice', 'LowPrice', 'OpenPrice', 'Volume', 'LastVolume', 'Turnover', 'OpenInt', 'PreOpenInt', 'OpenIntChg', 'ClosePrice', 'SetPrice', 'PreSetPrice', 'PreCloPrice', 'BuyVolume', 'SellVolume', 'AvgBuyPrice', 'AvgSellPrice', 'BidPrice1', 'BidPrice2', 'BidPrice3', 'BidPrice4', 'BidPrice5', 'AskPrice1', 'AskPrice2', 'AskPrice3', 'AskPrice4', 'AskPrice5', 'BidVolume1', 'BidVolume2', 'BidVolume3', 'BidVolume4', 'BidVolume5', 'AskVolume1', 'AskVolume2', 'AskVolume3', 'AskVolume4', 'AskVolume5', 'DerBidVolume1', 'DerBidVolume2', 'DerBidVolume3', 'DerBidVolume4', 'DerBidVolume5', 'DerAskVolume1', 'DerAskVolume2', 'DerAskVolume3', 'DerAskVolume4', 'DerAskVolume5', 'ULimitPrice', 'LLimitPrice', 'InstruCode', 'contract_id', 'date', 'contract_switch', 'DateTime', 'hour', 'minute', 'second', 'day_of_week', 'session', 'is_holiday', 'is_overnight', 'expiry_date', 'days_to_e

/Users/okko_dyd/Desktop/富国实习/factor_register.py:501: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  result['minute'] = pd.to_datetime(result['UpdateTime']).dt.minute



因子计算完成!
因子 intraday_seasonality 计算完成

开始计算因子: term_premium
数据列名: ['index', 'TradDay', 'UpdateTime', 'InstruID', 'LastPrice', 'HighPrice', 'LowPrice', 'OpenPrice', 'Volume', 'LastVolume', 'Turnover', 'OpenInt', 'PreOpenInt', 'OpenIntChg', 'ClosePrice', 'SetPrice', 'PreSetPrice', 'PreCloPrice', 'BuyVolume', 'SellVolume', 'AvgBuyPrice', 'AvgSellPrice', 'BidPrice1', 'BidPrice2', 'BidPrice3', 'BidPrice4', 'BidPrice5', 'AskPrice1', 'AskPrice2', 'AskPrice3', 'AskPrice4', 'AskPrice5', 'BidVolume1', 'BidVolume2', 'BidVolume3', 'BidVolume4', 'BidVolume5', 'AskVolume1', 'AskVolume2', 'AskVolume3', 'AskVolume4', 'AskVolume5', 'DerBidVolume1', 'DerBidVolume2', 'DerBidVolume3', 'DerBidVolume4', 'DerBidVolume5', 'DerAskVolume1', 'DerAskVolume2', 'DerAskVolume3', 'DerAskVolume4', 'DerAskVolume5', 'ULimitPrice', 'LLimitPrice', 'InstruCode', 'contract_id', 'date', 'contract_switch', 'DateTime', 'hour', 'minute', 'second', 'day_of_week', 'session', 'is_holiday', 'is_overnight', 'expiry_date', 'days_to_e

...


因子计算完成!
因子 intraday_seasonality 计算完成

开始计算因子: term_premium
数据列名: ['index', 'TradDay', 'UpdateTime', 'InstruID', 'LastPrice', 'HighPrice', 'LowPrice', 'OpenPrice', 'Volume', 'LastVolume', 'Turnover', 'OpenInt', 'PreOpenInt', 'OpenIntChg', 'ClosePrice', 'SetPrice', 'PreSetPrice', 'PreCloPrice', 'BuyVolume', 'SellVolume', 'AvgBuyPrice', 'AvgSellPrice', 'BidPrice1', 'BidPrice2', 'BidPrice3', 'BidPrice4', 'BidPrice5', 'AskPrice1', 'AskPrice2', 'AskPrice3', 'AskPrice4', 'AskPrice5', 'BidVolume1', 'BidVolume2', 'BidVolume3', 'BidVolume4', 'BidVolume5', 'AskVolume1', 'AskVolume2', 'AskVolume3', 'AskVolume4', 'AskVolume5', 'DerBidVolume1', 'DerBidVolume2', 'DerBidVolume3', 'DerBidVolume4', 'DerBidVolume5', 'DerAskVolume1', 'DerAskVolume2', 'DerAskVolume3', 'DerAskVolume4', 'DerAskVolume5', 'ULimitPrice', 'LLimitPrice', 'InstruCode', 'contract_id', 'date', 'contract_switch', 'DateTime', 'hour', 'minute', 'second', 'day_of_week', 'session', 'is_holiday', 'is_overnight', 'expiry_date', 'days_to_e

/Users/okko_dyd/Desktop/富国实习/factor_register.py:501: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  result['minute'] = pd.to_datetime(result['UpdateTime']).dt.minute



因子计算完成!
因子 intraday_seasonality 计算完成

开始计算因子: term_premium
数据列名: ['index', 'TradDay', 'UpdateTime', 'InstruID', 'LastPrice', 'HighPrice', 'LowPrice', 'OpenPrice', 'Volume', 'LastVolume', 'Turnover', 'OpenInt', 'PreOpenInt', 'OpenIntChg', 'ClosePrice', 'SetPrice', 'PreSetPrice', 'PreCloPrice', 'BuyVolume', 'SellVolume', 'AvgBuyPrice', 'AvgSellPrice', 'BidPrice1', 'BidPrice2', 'BidPrice3', 'BidPrice4', 'BidPrice5', 'AskPrice1', 'AskPrice2', 'AskPrice3', 'AskPrice4', 'AskPrice5', 'BidVolume1', 'BidVolume2', 'BidVolume3', 'BidVolume4', 'BidVolume5', 'AskVolume1', 'AskVolume2', 'AskVolume3', 'AskVolume4', 'AskVolume5', 'DerBidVolume1', 'DerBidVolume2', 'DerBidVolume3', 'DerBidVolume4', 'DerBidVolume5', 'DerAskVolume1', 'DerAskVolume2', 'DerAskVolume3', 'DerAskVolume4', 'DerAskVolume5', 'ULimitPrice', 'LLimitPrice', 'InstruCode', 'contract_id', 'date', 'contract_switch', 'DateTime', 'hour', 'minute', 'second', 'day_of_week', 'session', 'is_holiday', 'is_overnight', 'expiry_date', 'days_to_e

/Users/okko_dyd/Desktop/富国实习/factor_register.py:501: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  result['minute'] = pd.to_datetime(result['UpdateTime']).dt.minute



因子计算完成!
因子 intraday_seasonality 计算完成

开始计算因子: term_premium
数据列名: ['index', 'TradDay', 'UpdateTime', 'InstruID', 'LastPrice', 'HighPrice', 'LowPrice', 'OpenPrice', 'Volume', 'LastVolume', 'Turnover', 'OpenInt', 'PreOpenInt', 'OpenIntChg', 'ClosePrice', 'SetPrice', 'PreSetPrice', 'PreCloPrice', 'BuyVolume', 'SellVolume', 'AvgBuyPrice', 'AvgSellPrice', 'BidPrice1', 'BidPrice2', 'BidPrice3', 'BidPrice4', 'BidPrice5', 'AskPrice1', 'AskPrice2', 'AskPrice3', 'AskPrice4', 'AskPrice5', 'BidVolume1', 'BidVolume2', 'BidVolume3', 'BidVolume4', 'BidVolume5', 'AskVolume1', 'AskVolume2', 'AskVolume3', 'AskVolume4', 'AskVolume5', 'DerBidVolume1', 'DerBidVolume2', 'DerBidVolume3', 'DerBidVolume4', 'DerBidVolume5', 'DerAskVolume1', 'DerAskVolume2', 'DerAskVolume3', 'DerAskVolume4', 'DerAskVolume5', 'ULimitPrice', 'LLimitPrice', 'InstruCode', 'contract_id', 'date', 'contract_switch', 'DateTime', 'hour', 'minute', 'second', 'day_of_week', 'session', 'is_holiday', 'is_overnight', 'expiry_date', 'days_to_e

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from datetime import datetime
import gc
from factor_register import *
from factor_manager import *

# 导入我们的计算函数
from compute_daily_factors_and_returns import (
    compute_and_save_factors_with_returns,
    get_ic_summary,
    load_and_combine_processed_data
)
register_all_factors()
Manager = FactorManager()
# 设置参数
DATA_DIR = "output/volume_main"  # 主力合约数据目录
OUTPUT_DIR = "factors_ic_results"  # 结果保存目录
START_DATE = "2022-01-01"
END_DATE = "2025-12-31"
RETURN_PERIODS = [1, 5, 10, 20]  # 收益率周期

# 创建输出目录
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 定义要计算的因子
# 如果不指定，将计算所有注册的因子
# FACTOR_NAMES = None  # 设置为None表示计算所有因子
# 或者指定特定因子
FACTOR_NAMES = Manager.get_factor_names(frequency=FactorFrequency.TICK)

# 步骤1: 计算因子、收益率和IC值
print(f"开始计算因子和IC值 (日期范围: {START_DATE} 到 {END_DATE})...")
ic_summary = compute_and_save_factors_with_returns(
    data_dir=DATA_DIR,
    output_dir=OUTPUT_DIR,
    start_date=START_DATE,
    end_date=END_DATE,
    factor_names=FACTOR_NAMES,
    return_periods=RETURN_PERIODS,
    price_col="ClosePrice",  # 使用收盘价计算收益率
    dropna=True,
    overwrite=False,
    ic_method="spearman"  # 使用Spearman相关系数
)

# 步骤2: 获取并分析IC汇总数据
print("\n获取IC汇总数据...")
try:
    ic_df = get_ic_summary(OUTPUT_DIR)
    print(f"成功获取IC汇总数据，共有 {len(ic_df)} 条记录")
except FileNotFoundError:
    print("未找到IC汇总数据，请确保计算过程已完成")
    ic_df = None

# 步骤3: 可视化分析IC结果
if ic_df is not None:
    # 读取daily_ic.csv以获取每日的IC值
    daily_ic_path = os.path.join(OUTPUT_DIR, "daily_ic.csv")
    if os.path.exists(daily_ic_path):
        daily_ic = pd.read_csv(daily_ic_path)
        
        # 1. 将日期列转换为datetime
        if 'date' in daily_ic.columns:
            daily_ic['date'] = pd.to_datetime(daily_ic['date'])
        
        # 2. 计算每个因子的IC均值、IR值等指标
        ic_stats = []
        for col in daily_ic.columns:
            if col != 'date':
                ic_vals = daily_ic[col].dropna()
                if len(ic_vals) > 0:
                    ic_mean = ic_vals.mean()
                    ic_std = ic_vals.std()
                    ic_ir = ic_mean / ic_std if ic_std > 0 else np.nan
                    ic_abs_mean = abs(ic_vals).mean()
                    ic_gt_02 = (abs(ic_vals) > 0.02).mean()
                    ic_stats.append({
                        'factor': col,
                        'ic_mean': ic_mean,
                        'ic_std': ic_std,
                        'ic_ir': ic_ir,
                        'ic_abs_mean': ic_abs_mean,
                        'ic_rate_gt_02': ic_gt_02,
                        'count': len(ic_vals)
                    })
        
        if ic_stats:
            ic_stats_df = pd.DataFrame(ic_stats)
            ic_stats_df = ic_stats_df.sort_values('ic_abs_mean', ascending=False)
            
            # 保存IC统计结果
            ic_stats_df.to_csv(os.path.join(OUTPUT_DIR, "ic_statistics.csv"), index=False)
            print(f"IC统计结果已保存至 {os.path.join(OUTPUT_DIR, 'ic_statistics.csv')}")
            
  
            
            # 可视化前10个因子的IC分布
            plt.figure(figsize=(15, 8))
            top_factors = ic_stats_df['factor'].tolist()
            
            # 创建一个子图阵列
            fig, axes = plt.subplots(2, 5, figsize=(20, 10))
            axes = axes.flatten()
            
            for i, factor in enumerate(top_factors):
                if i < 10:  # 只显示前10个
                    sns.histplot(daily_ic[factor].dropna(), kde=True, ax=axes[i])
                    axes[i].set_title(f"{factor} (IR={ic_stats_df[ic_stats_df['factor']==factor]['ic_ir'].values[0]:.2f})")
                    axes[i].axvline(0, color='r', linestyle='--')
            
            plt.tight_layout()
            plt.savefig(os.path.join(OUTPUT_DIR, "top10_factors_ic_dist.png"))
            plt.close()
            
            # 可视化IC随时间的变化
            plt.figure(figsize=(15, 8))
            for factor in top_factors[:5]:  # 只显示前5个因子
                plt.plot(daily_ic['date'], daily_ic[factor], label=factor)
            
            plt.legend()
            plt.title("IC值随时间变化")
            plt.xlabel("日期")
            plt.ylabel("IC值")
            plt.grid(True)
            plt.tight_layout()
            plt.savefig(os.path.join(OUTPUT_DIR, "ic_time_series.png"))
            plt.close()
            
            print(f"\n图表已保存至 {OUTPUT_DIR} 目录")

print("\n计算完成! 结果保存在:", OUTPUT_DIR)

开始计算因子和IC值 (日期范围: 2022-01-01 到 2025-12-31)...
将计算指定的 31 个因子
找到 689 个符合条件的日期文件
将计算 1, 5, 10, 20 周期的收益率


处理日期:   0%|          | 1/689 [00:00<01:09,  9.85it/s]

处理日期: 20220104, 数据大小: (26586, 57)
计算 31 个Tick频率因子...

开始计算因子: volume_synchronized_probability, realized_vol_200, price_reversal, bid_ask_pressure, weighted_momentum_10, intraday_seasonality, effective_spread, order_flow_toxicity, order_book_imbalance, momentum_20, high_low_vol_100, hft_trend, weighted_momentum_20, weighted_momentum_100, realized_vol_50, amihud_illiquidity, high_low_vol_50, liquidity_adjusted_momentum, microstructure_momentum, term_premium, volume_price_trend, high_low_vol_200, momentum_100, price_impact, momentum_10, volume_intensity_100, quote_slope, weighted_momentum_50, vpin, momentum_50, realized_vol_100
数据列名: ['TradDay', 'UpdateTime', 'InstruID', 'LastPrice', 'HighPrice', 'LowPrice', 'OpenPrice', 'Volume', 'LastVolume', 'Turnover', 'OpenInt', 'PreOpenInt', 'OpenIntChg', 'ClosePrice', 'SetPrice', 'PreSetPrice', 'PreCloPrice', 'BuyVolume', 'SellVolume', 'AvgBuyPrice', 'AvgSellPrice', 'BidPrice1', 'BidPrice2', 'BidPrice3', 'BidPrice4', 'BidPrice5', 'AskPrice1', 'AskP

处理日期:   0%|          | 3/689 [00:00<01:00, 11.35it/s]


计算 31 个Tick频率因子...

开始计算因子: volume_synchronized_probability, realized_vol_200, price_reversal, bid_ask_pressure, weighted_momentum_10, intraday_seasonality, effective_spread, order_flow_toxicity, order_book_imbalance, momentum_20, high_low_vol_100, hft_trend, weighted_momentum_20, weighted_momentum_100, realized_vol_50, amihud_illiquidity, high_low_vol_50, liquidity_adjusted_momentum, microstructure_momentum, term_premium, volume_price_trend, high_low_vol_200, momentum_100, price_impact, momentum_10, volume_intensity_100, quote_slope, weighted_momentum_50, vpin, momentum_50, realized_vol_100
数据列名: ['TradDay', 'UpdateTime', 'InstruID', 'LastPrice', 'HighPrice', 'LowPrice', 'OpenPrice', 'Volume', 'LastVolume', 'Turnover', 'OpenInt', 'PreOpenInt', 'OpenIntChg', 'ClosePrice', 'SetPrice', 'PreSetPrice', 'PreCloPrice', 'BuyVolume', 'SellVolume', 'AvgBuyPrice', 'AvgSellPrice', 'BidPrice1', 'BidPrice2', 'BidPrice3', 'BidPrice4', 'BidPrice5', 'AskPrice1', 'AskPrice2', 'AskPrice3', 'AskPrice4',

...

处理日期:  20%|██        | 141/689 [00:10<00:46, 11.91it/s]

处理日期: 20220805, 数据大小: (26496, 57)
计算 31 个Tick频率因子...

开始计算因子: volume_synchronized_probability, realized_vol_200, price_reversal, bid_ask_pressure, weighted_momentum_10, intraday_seasonality, effective_spread, order_flow_toxicity, order_book_imbalance, momentum_20, high_low_vol_100, hft_trend, weighted_momentum_20, weighted_momentum_100, realized_vol_50, amihud_illiquidity, high_low_vol_50, liquidity_adjusted_momentum, microstructure_momentum, term_premium, volume_price_trend, high_low_vol_200, momentum_100, price_impact, momentum_10, volume_intensity_100, quote_slope, weighted_momentum_50, vpin, momentum_50, realized_vol_100
数据列名: ['TradDay', 'UpdateTime', 'InstruID', 'LastPrice', 'HighPrice', 'LowPrice', 'OpenPrice', 'Volume', 'LastVolume', 'Turnover', 'OpenInt', 'PreOpenInt', 'OpenIntChg', 'ClosePrice', 'SetPrice', 'PreSetPrice', 'PreCloPrice', 'BuyVolume', 'SellVolume', 'AvgBuyPrice', 'AvgSellPrice', 'BidPrice1', 'BidPrice2', 'BidPrice3', 'BidPrice4', 'BidPrice5', 'AskPrice1', 'AskP

处理日期:  21%|██        | 145/689 [00:11<00:43, 12.58it/s]

处理日期: 20220810, 数据大小: (25581, 57)
计算 31 个Tick频率因子...

开始计算因子: volume_synchronized_probability, realized_vol_200, price_reversal, bid_ask_pressure, weighted_momentum_10, intraday_seasonality, effective_spread, order_flow_toxicity, order_book_imbalance, momentum_20, high_low_vol_100, hft_trend, weighted_momentum_20, weighted_momentum_100, realized_vol_50, amihud_illiquidity, high_low_vol_50, liquidity_adjusted_momentum, microstructure_momentum, term_premium, volume_price_trend, high_low_vol_200, momentum_100, price_impact, momentum_10, volume_intensity_100, quote_slope, weighted_momentum_50, vpin, momentum_50, realized_vol_100
数据列名: ['TradDay', 'UpdateTime', 'InstruID', 'LastPrice', 'HighPrice', 'LowPrice', 'OpenPrice', 'Volume', 'LastVolume', 'Turnover', 'OpenInt', 'PreOpenInt', 'OpenIntChg', 'ClosePrice', 'SetPrice', 'PreSetPrice', 'PreCloPrice', 'BuyVolume', 'SellVolume', 'AvgBuyPrice', 'AvgSellPrice', 'BidPrice1', 'BidPrice2', 'BidPrice3', 'BidPrice4', 'BidPrice5', 'AskPrice1', 'AskP

处理日期:  21%|██▏       | 147/689 [00:11<00:42, 12.80it/s]

In [ ]:
import xgboost_signal
import pandas as pd
import tqdm
import factor_compute
import factor_register
import factor_manager
from sklearn.metrics import mean_squared_error
import factors_test
from factor_manager import *  # 确保导入FactorManager

data = pd.read_feather('data_with_factors.feather')
factor_register.register_all_factors()
Manager = FactorManager()
factors_list = Manager.get_factor_names(frequency=FactorFrequency.TICK)
factors_list = [factor for factor in factors_list if 'vpin' not in factor]
start_date = pd.to_datetime('2024-04-22')
end_date = pd.to_datetime('2024-11-18')
results = xgboost_signal.generate_rolling_signals(
    df, 
    traget='mid_price',
    feature_cols=factors_list,
    forward_periods=10,  # 10个周期后的收益率
    min_train_samples=1000)
predictions = results['predictions']
df_with_predictions = df.copy()
df_with_predictions['xgboost_pred'] = predictions
df_with_predictions.to_feather("xgboost_results/data_with_predictions.feather")

# 显示模型评估指标
print("\n\n===== 模型评估结果汇总 =====")
metrics_df = results['metrics']
print("\n时间窗口评估指标均值:")
mean_metrics = metrics_df.mean(numeric_only=True)
for metric, value in mean_metrics.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
import pandas as pd
import numpy as np
from factor_register import register_all_factors
from factor_manager import FactorManager, FactorFrequency
from factors_test import FactorsTester, FactorTestConfig
import matplotlib.pyplot as plt
import seaborn as sns

# 加载数据
print("加载数据...")
df = pd.read_feather("data_with_factors.feather")

# 注册因子
register_all_factors()
Manager = FactorManager()
all_factors = Manager.get_factor_names(frequency=FactorFrequency.TICK)

# 筛选要调整的因子 (排除vpin)
factors_to_adjust = [factor for factor in all_factors if 'vpin' not in factor]
print(f"\n将调整 {len(factors_to_adjust)} 个因子的符号")

# 计算前向收益率
return_periods = [10]
df_with_returns = FactorsTester.calculate_forward_returns(df, periods=return_periods)

# 定义一个简化版的调整因子符号函数
def adjust_signs(data, factor_names, period=10):
    result_df = data.copy()
    adjusted_factors = []
    
    for factor_name in factor_names:
        if factor_name not in result_df.columns:
            print(f"警告: 因子 {factor_name} 不在数据中，跳过")
            continue
            
        try:
            # 计算相关系数 (IC)直接使用spearman相关系数
            valid_data = result_df[[factor_name, f'{period}period_return']].dropna()
            if len(valid_data) < 30:  # 最小样本要求
                print(f"警告: 因子 {factor_name} 有效样本不足，跳过")
                continue
                
            ic = valid_data[factor_name].corr(valid_data[f'{period}period_return'], method='spearman')
            
            # 如果IC为负，调整因子符号
            if ic < 0:
                result_df[factor_name] = -result_df[factor_name]
                print(f"调整因子 {factor_name} 的符号 (IC: {ic:.4f})")
                adjusted_factors.append(factor_name)
            else:
                print(f"因子 {factor_name} 无需调整 (IC: {ic:.4f})")
                
        except Exception as e:
            print(f"调整因子 {factor_name} 符号时出错: {str(e)}")
            continue
    
    print(f"\n调整完成! 共调整了 {len(adjusted_factors)} 个因子的符号")
    if adjusted_factors:
        print("调整的因子列表:")
        print(", ".join(adjusted_factors))
        
    return result_df

# 调整因子符号
print("\n开始调整因子符号...")
df_adjusted = adjust_signs(df_with_returns, factors_to_adjust, period=10)

# 计算相关性矩阵
print("\n计算相关性矩阵...")
# 只选择已调整的因子列
available_factors = [f for f in factors_to_adjust if f in df_adjusted.columns]
correlation_matrix = df_adjusted[available_factors].corr()

# 保存相关性矩阵
correlation_matrix.to_csv("factors_correlation_adjusted.csv")
print("相关性矩阵已保存至 factors_correlation_adjusted.csv")

# 可视化相关性矩阵
plt.figure(figsize=(20, 16))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=False, cmap='coolwarm', 
            vmin=-1, vmax=1, square=True, linewidths=.5)
plt.title('因子相关性矩阵 (调整符号后)', fontsize=16)
plt.tight_layout()
plt.savefig('factors_correlation_adjusted.png', dpi=300)
print("相关性矩阵图已保存至 factors_correlation_adjusted.png")

# 保存调整后的数据
df_adjusted.to_feather("data_with_factors_adjusted.feather")
print("\n调整后的数据已保存至 data_with_factors_adjusted.feather")

In [ ]:
import pandas as pd
data = pd.read_feather('data_with_factors_adjusted.feather')
factor_list = data.columns
factor_list